In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import leastsq, minimize
from numpy.linalg import inv
import matplotlib.pyplot as plt

# Reading in the data
# For some reason no matter what I do, I lose precision
# Also did my own formatting of the data to make it pandas readable
dt = pd.read_table('data.txt',sep=' ')




/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [37]:
'''
Part A
'''

dt.plot.scatter('x','y',yerr='yerr',color='black')
plt.ylim(0,2.5)
plt.savefig('plots/p2_a.png')
plt.show()



In [38]:
'''
Part B
'''

# Defining data
x = dt['x']
y = dt['y']
yerr = dt['yerr']

# Initial guess
b0 = np.array([3,2,1.5,4,2])

# Defining the model function
def model(x,b):
        return b[0]*np.exp(-b[1]*x)+b[2]*np.exp(-0.5*(x-b[3])**2/b[4]**2)
def chi_square(b):
        return np.sum(((y-model(x,b))/yerr)**2)

# Running optimization
b_opt = minimize(chi_square,b0)

# Running model with parameters
y_model = model(x,b_opt.x)

# Plotting
dt.plot.scatter('x','y',yerr='yerr',color='black',label='Data')
plt.plot(x,y_model,color='green',label='Model')
plt.legend(loc='best')
plt.savefig('p2_b.png')
plt.ylim(0,2.5)
plt.show()

print('These are the best fit parameter values: ',b_opt.x)


These are the best fit parameter values:  [ 2.02401118  1.37622564  1.17978525  4.09101789  1.82923263]


In [39]:
'''
Part C
'''

# Scipy minimize returns the inverse Hessian, so to obtain the Hessian, all we have to do is use
# Numpy's inverse function

hess = b_opt.hess_inv
hess = inv(hess)

covar = inv(0.5*hess)

print('This is the Hessian',hess)
print('This is the covariance',covar)



This is the Hessian [[  163.01764563  -193.0129768    148.02759187  -129.22906275
    200.09654948]
 [ -193.0129768    373.08361084  -436.21921854   326.45012634
   -442.67836423]
 [  148.02759187  -436.21921854  2323.94425319  -100.67154137  1183.2042628 ]
 [ -129.22906275   326.45012634  -100.67154137   762.29592102
   -100.51325096]
 [  200.09654948  -442.67836423  1183.2042628   -100.51325096
   1017.63445442]]
This is the covariance [[ 0.03942009  0.03357584  0.00085689 -0.00690023  0.00517675]
 [ 0.03357584  0.05769213 -0.0014923  -0.01676243  0.01857395]
 [ 0.00085689 -0.0014923   0.00229831  0.00063598 -0.00342708]
 [-0.00690023 -0.01676243  0.00063598  0.00793965 -0.00589023]
 [ 0.00517675  0.01857395 -0.00342708 -0.00589023  0.01243013]]


In [27]:
'''
Part D
'''
def sigma(array):
        '''
        This function finds the individual errors of the parameters 
        using the covariance array
        '''
        sigma = []
        for i in range(array.shape[0]):
                sigma.append(np.sqrt(array[i][i]))
        return np.array(sigma)


err = sigma(covar)
print(err)



[ 0.19854492  0.24019187  0.04794073  0.08910473  0.11149051]


In [28]:
'''
Part E
'''
def marginalize(hess,covar,para_1,para_2):
        '''
        Will marginalize all the parameters except the ones inputted
        and return the NxN hessian and covariance of inputted parameters
        '''
        p1 = para_1 - 1
        p2 = para_2 - 1

        h_ii = hess[p1][p1]
        h_ij = hess[p1][p2]
        h_ji = hess[p2][p1]
        h_jj = hess[p2][p2]
        
        c_ii = covar[p1][p1]
        c_ij = covar[p1][p2]
        c_ji = covar[p2][p1]
        c_jj = covar[p2][p2]

        marg_hess = np.array([[h_ii,h_ij],[h_ji,h_jj]])
        marg_covar = np.array([[c_ii,c_ij],[c_ji,c_jj]])
        
        return marg_hess, marg_covar

hess_m, covar_m = marginalize(hess,covar,3,5)

# Sigma values for b3 and b5 are the same from
# Equation for a an ellipse that outlines the 95% 
# (b3/sigma_b3)**2 + (b5/sigma_b5)**2 = 5.991
# the 5.991 comes from a chi**2 distribution for 2 DOF

In [75]:
alpha

1.2734069009890041

In [74]:
from numpy.linalg import eig
from matplotlib.patches import Ellipse
import math
# Finds the eigen values to calculate the angle
eigen_val, eigen_vec = eig(covar_m)

eigen_max = np.where(eigen_val == np.max(eigen_val))[0][0]

# calculates the angle
alpha = np.arctan(eigen_vec[eigen_max][1]/eigen_vec[eigen_max][0])

# calculating length of major and minor axis
minor = 2*np.sqrt(eigen_val[0]*5.991)
major = 2*np.sqrt(eigen_val[1]*5.991)

# placing x and y values
x = b_opt.x[2] # b3
y = b_opt.x[4] # b4

print(alpha)


#print(eigen_val)
# Plots it all 
ell = Ellipse(xy=(x,y), width=minor, height=major, angle=alpha(180/math.pi))
fig,ax = plt.subplots(nrows=1,ncols=1)
ax.add_artist(ell)
plt.show()


1.27340690099


TypeError: 'numpy.float64' object is not callable

# Question 3

In [40]:
'''
Part A
'''

def LPE(array,param_1,param_2):
    '''
    Takes in the 2x2 marginalized covariance matrix, parameter 1's value, and parameter 2' value 
    and calculates the error using Linearized propagation of errors
    '''
    var = (param_1**2*array[0][0])+(2*param_1*param_2*array[0][1])+(param_2**2*array[1][1])
    sigma = np.sqrt(var)
    
    return sigma

f = b_opt.x[2]*b_opt.x[4]
sigma = LPE(covar_m,b_opt.x[4],b_opt.x[2])

print(f,'+/-',sigma)

2.1581016683 +/- 0.100994049028


In [46]:
var = b_opt.x[2]**2*covar[4][4] + 2*b_opt.x[2]*b_opt.x[4]*covar[2][4] + b_opt.x[4]**2*covar[2][2]
sigma = np.sqrt(var)

print(covar)
print(var)
print(sigma)


[[ 0.03942009  0.03357584  0.00085689 -0.00690023  0.00517675]
 [ 0.03357584  0.05769213 -0.0014923  -0.01676243  0.01857395]
 [ 0.00085689 -0.0014923   0.00229831  0.00063598 -0.00342708]
 [-0.00690023 -0.01676243  0.00063598  0.00793965 -0.00589023]
 [ 0.00517675  0.01857395 -0.00342708 -0.00589023  0.01243013]]
0.010199797939
0.100994049028


In [51]:
'''
Part B
'''

bees = np.random.multivariate_normal(b_opt.x,covar,10**4)
humps = bees.T[2]*bees.T[4]
plt.hist(humps,bins=30)
plt.savefig('plots/p3_b.png')
plt.show()
std = np.std(humps)
mean = np.mean(humps)

print(mean,'+/-',std)

2.1534494435 +/- 0.10088058284
